# Prepare Data for Map Matching

In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd
from leuvenmapmatching.matcher.distance import DistanceMatcher
from leuvenmapmatching.map.inmem import InMemMap
#from leuvenmapmatching import visualization as mmviz
import pickle
import time
import datetime
from pathlib import Path

from tqdm import tqdm
from shapely.ops import Point, LineString
import matplotlib.pyplot as plt

from importlib import reload

from bikewaysim.paths import config
from bikewaysim import map_match
from bikewaysim.network import prepare_network

# import network
with (config['matching_fp'] / 'map_con.pkl').open('rb') as fh:
    exploded_links, exploded_nodes = pickle.load(fh)
map_con = map_match.make_network(exploded_links,exploded_nodes)
link_types_allowed = exploded_links['link_type'].unique().tolist()

idx = 5
with (config['matching_fp'] / f'coords_dict_{idx}.pkl').open('rb') as fh:
    coords_dict = pickle.load(fh)

with (config['matching_fp']/'match_settings.pkl').open('rb') as fh:
    matching_settings = pickle.load(fh)

match_dict = {tripid:map_match.leuven_match(trace,matching_settings,map_con,exploded_links) for tripid, trace in tqdm(coords_dict.items(),total=len(coords_dict))}

# Now match_dict contains the processed results
with (config['matching_fp'] / f'match_dict_{idx}.pkl').open('wb') as fh:
    pickle.dump(match_dict,fh)